In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import datetime
from bs4 import BeautifulSoup
import time, os
import pandas as pd

In [ ]:
driver = webdriver.Chrome()

In [ ]:
MY_WORK_SPACE = 'XXXXX'
driver.get(f"https://app.azendoo.com/#workspaces/{MY_WORK_SPACE}/conversations")
# login manually !

In [ ]:
# scroll down (as long as needed)
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
for i in range(0, 10):
    html = driver.find_element(By.CLASS_NAME, 'fr-FR')
    html.send_keys(Keys.END)
    time.sleep(0.75)
    if i%10==0:
        print(i, end=',')
        # display full messages
        
        for e in driver.find_elements(By.CLASS_NAME, 'u-text-small'):
            if e.text in ['Afficher la suite']:
                e.click()
                time.sleep(0.1)
                
        for e in driver.find_elements(By.LINK_TEXT, 'Afficher les commentaires précédents')
            e.click()
            time.sleep(0.1)

In [ ]:
conversations = driver.find_elements(By.CLASS_NAME, 'conversations-item')
len(conversations)

In [ ]:
def parse_conversation(conv, current_id, current_tab):
    
    if '000000000' in conv.text:
        return None

    author = conv.find_element(By.CLASS_NAME, 'conversations-item-author').text
    try:
        canal = conv.find_element(By.CLASS_NAME, 'link--action').text
    except:
        canal = 'Général'
    try:
        date = conv.find_element(By.CLASS_NAME, 'conversations-informations-date').text
    except:
        date = '2017-01-01'
    try:
        body = conv.find_element(By.CLASS_NAME, 'conversations-message-body').text
    except:
        return None
    
    elt = {'author': author, 'canal': canal, 'date': date, 'body': body}
    
    pjs, images, comments = None, None, None
    pieces, img, all_comments = [], [], []
    try:
        pjs = conv.find_elements(By.CLASS_NAME, 'medias-list-simple-item-title') 
    except:
        pass
    if pjs:
        for pj in pjs:
            pj_title = pj.text
            url = pj.get_attribute('href')
            internal_title = f'{current_id}_{pj_title}'
           
            pieces.append({'pj_title': pj_title, 'file':internal_title, 'url': url})
        elt['pj'] = pieces
            
    try:
        images = conv.find_elements(By.CLASS_NAME, 'medias-list-simple-item-image-preview')  
    except:
        pass
    if images:
        for image in images:
            try:
                image_url = image.get_attribute('style').split(';')[0].split('"')[1]
                img.append({'url': image_url})
            except:
                pass
        elt['images'] = img
        #print(elt)
        
    try:
        comments = conv.find_elements(By.CLASS_NAME, 'comments-content')
    except:
        pass
    if comments:
        for comment in comments:
            try:
                author = comment.find_element(By.CLASS_NAME, 'user-link').text
                text = comment.find_element(By.CLASS_NAME, 'conversations-body').text
                dt = comment.find_element(By.CLASS_NAME, 'text-muted').text
                if text:
                    all_comments.append({'author': author, 'text': text, 'date': dt[2:]})
            except:
                pass
        if all_comments:
            elt['comments'] = all_comments
        
    
    return elt

In [ ]:
data_general = []
current_tab = driver.window_handles[0]
for current_id, conv in enumerate(conversations):
    
    if current_id % 10 == 0:
        print(current_id, end=',')
    #try:
    elt = parse_conversation(conv, f'general_{current_id}', current_tab)
    #except:
    #    elt = None
    
    #print(elt)
    if elt:
        data_general.append(elt)
    

In [ ]:
pd.DataFrame(data_general).to_json('data_azendoo.jsonl', lines=True, orient='records')

In [ ]:
import requests
def download_file(URL, target):
    response = requests.get(URL)
    open(target, "wb").write(response.content)

In [ ]:
# download images
import os
os.mkdir('pj')

for d in data_general:
    for pj in d.get('images', []):
        download_file(pj['url'], f"pj/{pj['url'].split('/')[-1]}")

In [ ]:
# download attachments
for d in data_general:
    for pj in d.get('pj', []):
        try:
            download_file(pj['url'], f"pj/{pj['file'].split('/')[-1]}")
        except:
            print(pj['url'])

In [ ]:
d